# Loading and Pre-processing Student Performance dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random,time,csv
import math,copy,os
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
import six.moves.urllib as urllib
from google.protobuf import text_format
from sklearn import preprocessing
import tensorflow as tf
tf.compat.v1.enable_v2_behavior()
! pip install fairlearn
import random
import random,time,csv
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.svm import SVC
import os
import tempfile
import pandas as pd
import six.moves.urllib as urllib
import pprint
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from google.colab import files
from fairlearn.reductions import BoundedGroupLoss, ZeroOneLoss
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.insert(1, '/content/drive/Shareddrives/622 - Privacy/Project Codebase/Fair-SMOTE-master/')
from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy
from Generate_Samples import generate_samples

     |████████████████████████████████| 177 kB 7.1 MB/s 


In [ ]:
# loading the pre-processed dataset
df = pd.read_csv('/content/drive/Shareddrives/622 - Privacy/Project Codebase/CSV Data/student_data.csv')
df.head(3)

,isMale,urban_resident,studytime,failures,activities,nursery,higher,internet,romantic,freetime,goout,Dalc,Walc,health,absences,grade
0,0,1,2,0,0,1,1,0,0,3,4,1,1,3,4,11
1,0,1,2,0,0,0,1,1,0,3,3,1,1,3,2,11
2,0,1,2,0,0,1,1,1,0,3,2,2,3,3,6,12


In [ ]:
# creating a new binary column 'above_average_threshold' based on median of the grade column to replicate Fair-SMOTE methodology 
# also removing any null values
median_grade = df["grade"].median()
threshold = median_grade
print(' median grade is' ,threshold)
df['grade_above_threshold'] = df.apply(lambda row: 1 if row['grade'] > threshold else 0, axis=1)
df = df.dropna()

 median grade is 12.0


In [ ]:
# scaling the dataset
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df),columns = df.columns)

In [ ]:
# storing a copy to use later for Linear Regression on original dataset
data_original = df


In [ ]:
# storing scaled average grade to be used for Statistical Parity 
threshold = df["grade"].mean()

# Oversampling and removing biased labels using FairSMOTE

## Oversampling

In [ ]:
# first one is class value and second one is protected attribute value
# isMale was considered as the PROTECTED ATTRIBUTE 
male_ab_av = len(df[(df['grade_above_threshold'] == 1) & (df['isMale'] == 1)])
male_bl_av= len(df[(df['grade_above_threshold'] == 0) & (df['isMale'] == 1)])
female_ab_av= len(df[(df['grade_above_threshold'] == 1) & (df['isMale'] == 0)])
female_bl_av= len(df[(df['grade_above_threshold'] == 0) & (df['isMale'] == 0)])

print(' Instance distribution -- ',male_ab_av,male_bl_av,female_ab_av,female_bl_av)

 Instance distribution --  93 173 183 200


In [ ]:
maximum = max(male_ab_av,male_bl_av,female_ab_av,female_bl_av)

male_ab_av_to_be_incresed = maximum - male_ab_av
male_bl_av_to_be_incresed = maximum - male_bl_av 
female_ab_av_to_be_incresed = maximum - female_ab_av

#Female students with grade < average are more 
# Therefore, oversampling other subgroups
print(' Instances to be increased by --',male_ab_av_to_be_incresed,male_bl_av_to_be_incresed,female_ab_av_to_be_incresed)

 Instances to be increased by -- 107 27 17


In [ ]:
# generating samples and making instance counts equal
df_male_ab_av = (df[(df['grade_above_threshold'] == 1) & (df['isMale'] == 1)])
df_male_bl_av= (df[(df['grade_above_threshold'] == 0) & (df['isMale'] == 1)])
df_female_ab_av= (df[(df['grade_above_threshold'] == 1) & (df['isMale'] == 0)])

# using Generate samples function given by FairSMOTE authors. It uses KNN

df_male_ab_av = generate_samples(male_ab_av_to_be_incresed,df_male_ab_av,'law').rename(columns={0:'isMale',1: 'urban_resident', 2:'studytime',3: 'failures', 4:'activities',5:'nursery',6: 'higher', 7:'internet',8: 'romantic', 9:'freetime',10: 'goout',11:'Dalc', 12:'Walc',13: 'health',14: 'absences',15: 'grade', 16:'grade_above_threshold'})
df_male_bl_av= generate_samples(male_bl_av_to_be_incresed,df_male_bl_av,'law').rename(columns={0:'isMale',1: 'urban_resident', 2:'studytime',3: 'failures', 4:'activities',5:'nursery',6: 'higher', 7:'internet',8: 'romantic', 9:'freetime',10: 'goout',11:'Dalc', 12:'Walc',13: 'health',14: 'absences',15: 'grade', 16:'grade_above_threshold'})
df_female_ab_av= generate_samples(female_ab_av_to_be_incresed,df_female_ab_av,'law').rename(columns={0:'isMale',1: 'urban_resident', 2:'studytime',3: 'failures', 4:'activities',5:'nursery',6: 'higher', 7:'internet',8: 'romantic', 9:'freetime',10: 'goout',11:'Dalc', 12:'Walc',13: 'health',14: 'absences',15: 'grade', 16:'grade_above_threshold'})


In [ ]:
# combining all the oversampled subgroups 

df_female_bl_av= (df[(df['grade_above_threshold'] == 0) & (df['isMale'] == 0)])
data_oversampled = df_female_bl_av.append([df_male_ab_av,df_male_bl_av,df_female_ab_av])

print('The oversampled data has shape --',data_oversampled.shape)

The oversampled data has shape -- (800, 17)


## Removal of biased data points using situation testing


In [ ]:
import random 

data_oversampled['new_index'] = range(0,800) # FairSMOTE implementation repeats indexes according to the subgrouped dfs
data_oversampled = data_oversampled.set_index('new_index')
X_train, y_train = data_oversampled.loc[:, data_oversampled.columns != 'grade_above_threshold'], data_oversampled['grade_above_threshold']


clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf.fit(X_train,y_train)
removal_list = []

for index,row in X_train.iterrows():
    row_ = [row.values[0:len(row.values)]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: # index of isMale is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        removal_list.append(index)


Note : CANNOT CHANGE A 0 VALUE TO 1, AS IT IS ONE-HOT ENCODED, IT CAN BE CHANGED TO 1 MAYBE : GOTTA DECIDE ON THIS; right now, ill change, lets say, the value from asian to any other race randomly;

In [ ]:
removal_list = list(set(removal_list))
print(f' A total of {len(removal_list)} biased datapoints will be removed')

 A total of 39 biased datapoints will be removed


In [ ]:
# removing biased datapoints are generating final report
final = data_oversampled.drop(data_oversampled.index[list(set(removal_list))])
print(f' Final dataset has {len(final)} instances')

 Final dataset has 761 instances


In [ ]:
# saving the final dataset
final.to_csv('/content/drive/Shareddrives/622 - Privacy/Project Codebase/CSV Data/fairSMOTE_oversampled_student_data.csv')

# Evaluation

##  Linear Regression  on oversampled data

In [ ]:
# training a linear regression model which will be used for evaluation
# Regression with Protected attribute Gender 
data_oversampled = final
data_oversampled_onlygrade = data_oversampled.drop('grade_above_threshold',axis = 1) # removing binary column that was generated

df = data_oversampled_onlygrade

X = df.iloc[:,df.columns != 'grade']
Y = df['grade']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred_w_prot_attr = model.predict(X_test)




rmsd = np.sqrt(mean_squared_error(Y_test, y_pred_w_prot_attr))      
r2_value = r2_score(Y_test, y_pred_w_prot_attr)                     

print("Intercept: ", model.intercept_)
print("Root Mean Square Error: ", rmsd)
print("R^2 Value: ", r2_value)

Intercept:  0.5519155752899263
Root Mean Square Error:  0.16052838973131836
R^2 Value:  0.18540639725299946


In [ ]:
# without protected attribute

_df = data_oversampled_onlygrade.drop(['isMale'], axis = 1)
X = _df.iloc[:,_df.columns != 'grade']
Y = _df['grade']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred_wo_prot_attr = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred_wo_prot_attr))      
r2_value = r2_score(Y_test, y_pred_wo_prot_attr)                     

print(type(Y_test))

print(type(y_pred_wo_prot_attr))


print("Intercept: ", model.intercept_)
print("Root Mean Square Error: ", rmsd)
print("R^2 Value: ", r2_value)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
Intercept:  0.5525072503598543
Root Mean Square Error:  0.1597970155453922
R^2 Value:  0.19281213465660363


In [ ]:
# Statistical Parity

counter = 0
for entry in y_pred_w_prot_attr:
  if entry > threshold : counter+=1
prob_w_prot_attr = counter/len(y_pred_w_prot_attr)

counter = 0
for entry in y_pred_wo_prot_attr:
  if entry > threshold : counter+=1
prob_wo_prot_attr = counter/len(y_pred_wo_prot_attr)

print('Statistical Parity is ', abs(prob_w_prot_attr-prob_wo_prot_attr))


Statistical Parity is  0.006535947712418277


In [ ]:
# Bounded Group Loss 

X = df.iloc[:,df.columns != 'grade']
Y = df['grade']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(list(X_test['isMale'].values))
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred_w_prot_attr,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print(mae_frame.overall)

0.11682101122631677


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)
